In [ ]:
# Simple AutoEncoder (no convolutional layers)
# https://blog.keras.io/building-autoencoders-in-keras.html
# https://www.youtube.com/watch?v=wECwVBmPH7w

https://www.tensorflow.org/guide/basics

# Learned Image Compression (LIC) using autoencoders

For reasons of space and speed, we use the mnist dataset, that is composed of 70K images of hand written decimal digits. Notice that with this dataset, all the images have the same shape (28x28 pixels), and are monochromatic (8 bits/pixel). We will use [Keras](https://blog.keras.io/building-autoencoders-in-keras.html).

## 1. Using 1 hidden layer and no quantization of the latent space

Only the latent layer is hidden.

    Input -> Latent -> Output
    28x28    32        28x28

In [ ]:
# Autoencoding example (feeding tf.data.Dataset with NumPy arrays, iterative)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize

print(tf.__version__)

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['x_train']
    x_test = data['x_test']
    y_test = data['x_test']

# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_DS = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Define how to use the datasets, preprocess, etc.

def process_x(image, label):
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, label

def process_y(image, label):
    # The last layer of the network is a dense layer and TensorFlow imposes 1D.
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(process_x)
train_DS = train_DS.map(process_y)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(process_x)
test_DS = test_DS.map(process_y)
test_DS = test_DS.batch(BATCH_SIZE)

# Define the autoencoder.
input_layer = keras.Input(
    shape=(img_length,))
hidden_layer = layers.Dense(
    LATENT_SPACE_LENGTH,
    activation="relu")(input_layer)
output_layer = layers.Dense(
    img_length,
    activation="sigmoid")(hidden_layer)

autoencoder = keras.Model(
    input_layer,
    output_layer)

# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(
    input_layer,
    hidden_layer)

autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

autoencoder.summary()

# Train.
EPOCHS = 1
ITERATIONS = 5
plt.gray()
for i in range(ITERATIONS):

    autoencoder.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    # Show the learning.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    # Show the originals (using the pipeline).
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(3, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                #plt.gray()
                plt.axis("off")
            except:
                pass
    
    # See the latent space (as images).
    latent_imgs = encoder.predict(test_DS)
    print("type latent", type(latent_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
        plt.axis("off")    
    
    # See the reconstructions.
    reconstructed_imgs = autoencoder.predict(test_DS)
    print(type(reconstructed_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.axis("off")    

    plt.show()

##  2. Quantizing the latent space
Add a new custom layer that quantizes the latent space.

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['x_train']
    x_test = data['x_test']
    y_test = data['x_test']
    
# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_DS = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Define how to use the datasets, preprocess, etc.

def process_x(image, label):
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, label

def process_y(image, label):
    # The last layer of the network is a dense layer and TensorFlow imposes 1D.
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(process_x)
train_DS = train_DS.map(process_y)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(process_x)
test_DS = test_DS.map(process_y)
test_DS = test_DS.batch(BATCH_SIZE)

class Quantization_8bits(keras.layers.Layer):

    def __init__(self, units=32, **kwargs):
        super(Quantization_8bits, self).__init__(**kwargs)
        self.quantized_vals = 0

    def call(self, inputs):
        outputs = inputs
        outputs = tf.saturate_cast(tf.round(outputs * 255.), tf.int8)
        self.quantized_vals = outputs
        outputs = tf.cast(outputs, tf.float32) / 255.0
        return outputs
    
    def get_vals(self):
        return self.quantized_vals

# Define the autoencoder.
input_layer = keras.Input(
    shape=(img_length,),
    name="inputs")
hidden_layer = layers.Dense(
    LATENT_SPACE_LENGTH,
    activation="relu")(input_layer)
quantization_layer = Quantization_8bits(
    units=LATENT_SPACE_LENGTH,
    name="quantization"
    )(hidden_layer)
output_layer = layers.Dense(
    img_length,
    activation="sigmoid",
    name="outputs")(quantization_layer)

autoencoder = keras.Model(
    input_layer,
    output_layer)

# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(
    input_layer,
    hidden_layer)

autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy", run_eagerly=True
)

autoencoder.summary()

# Train.
EPOCHS = 1
ITERATIONS = 1
plt.gray()
for i in range(ITERATIONS):

    autoencoder.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    batch_vals = autoencoder.get_layer("quantization").get_vals()
    print("batch_vals =", batch_vals)
    for j in range(BATCH_SIZE):
        try:
            vals = batch_vals[j].numpy() + 512
            #print(vals)
            print(entropy(vals, base=2))
        except:
            pass
    
    #print("2", autoencoder.get_layer("hidden").variables[1])

    # Show the learning.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    # Show the originals (using the pipeline).
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(3, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                #plt.gray()
                plt.axis("off")
            except:
                pass
    
    # See the latent space (as images).
    latent_imgs = encoder.predict(test_DS)
    print("type latent", type(latent_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
        plt.axis("off")    
    
    # See the reconstructions.
    reconstructed_imgs = autoencoder.predict(test_DS)
    print(type(reconstructed_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.axis("off")    

    plt.show()

##  3. Quantizing the latent space (only during inferencing)
During training, the quantization noise is simulated. During the inference, the latents are quantized.

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['x_train']
    x_test = data['x_test']
    y_test = data['x_test']
    
# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_DS = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Define how to use the datasets, preprocess, etc.

def process_x(image, label):
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, label

def process_y(image, label):
    # The last layer of the network is a dense layer and TensorFlow imposes 1D.
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(process_x)
train_DS = train_DS.map(process_y)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(process_x)
test_DS = test_DS.map(process_y)
test_DS = test_DS.batch(BATCH_SIZE)

class AddNoise(keras.layers.Layer):

    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        outputs = inputs
        outputs += tf.random.uniform(shape=tf.shape(inputs), minval=-.5, maxval=.5)
        return outputs

class Quantization_8bits(keras.layers.Layer):

    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.quantized_vals = 0

    def call(self, inputs):
        outputs = inputs
        #print("a1", tf.reduce_min(outputs))
        #print("a2", tf.reduce_max(outputs))
        #outputs = tf.keras.activations.sigmoid(outputs - 0.5)#(outputs + 0.5)
        #print("d1", tf.reduce_min(outputs))
        #print("d2", tf.reduce_max(outputs))
        #output = outputs - 0.5
        outputs = tf.saturate_cast(tf.round(outputs * 255.), tf.int16)
        self.quantized_vals = outputs
        outputs = tf.cast(outputs, tf.float32) / 255.0
        return outputs
    
    def get_vals(self):
        return self.quantized_vals

# Define the autoencoder.
input_layer = keras.Input(
    shape=(img_length,),
    name="inputs")
hidden_layer = layers.Dense(
    LATENT_SPACE_LENGTH,
    activation="relu")(input_layer)
noisy_layer = AddNoise(
    units=LATENT_SPACE_LENGTH,
    name="add_noise"
    )(hidden_layer)
output_layer = layers.Dense(
    img_length,
    activation="sigmoid",
    name="outputs")(noisy_layer)

autoencoder = keras.Model(
    input_layer,
    output_layer)

# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(
    input_layer,
    hidden_layer)

autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy", run_eagerly=True
)

autoencoder.summary()

input_layer_2 = keras.Input(
    shape=(img_length,),
    name="inputs_2")
hidden_layer_2 = layers.Dense(
    LATENT_SPACE_LENGTH,
    activation="relu")(input_layer_2)
quantization_layer = Quantization_8bits(
    units=LATENT_SPACE_LENGTH,
    name="quantization"
    )(hidden_layer_2)
output_layer_2 = layers.Dense(
    img_length,
    activation="sigmoid",
    name="outputs_2")(quantization_layer)

autoencoder_2 = keras.Model(
    input_layer_2,
    output_layer_2
)

autoencoder_2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy", run_eagerly=True
)

autoencoder_2.summary()

# Train.
EPOCHS = 1
ITERATIONS = 1
plt.gray()
for i in range(ITERATIONS):

    autoencoder.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    #print("2", autoencoder.get_layer("hidden").variables[1])

    # Show the learning.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    # Show the originals (using the pipeline).
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(3, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                #plt.gray()
                plt.axis("off")
            except:
                pass
    
    # See the latent space (as images).
    latent_imgs = encoder.predict(test_DS)
    print("type latent", type(latent_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
        plt.axis("off")    
    
    # See the reconstructions.
    autoencoder_2.set_weights(autoencoder.get_weights())
    reconstructed_imgs = autoencoder_2.predict(test_DS)
    print(type(reconstructed_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.axis("off")
        
    batch_vals = autoencoder_2.get_layer("quantization").get_vals()
    print("batch_vals =", batch_vals)
    for j in range(BATCH_SIZE):
        try:
            vals = batch_vals[j].numpy() + 512
            #print(vals)
            print(entropy(vals, base=2))
        except:
            pass
    
    plt.show()

##  4. Considering RD tradeoff
Optimize the autoencoder for a given $J = R + \lambda D$.

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
#from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

train_DS, test_DS = tfds.load(
    DATASET,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)

def encode(latent_dims):
  """Creates the analysis (encoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Input(
            shape=(img_length,), name="inputs"),
      tf.keras.layers.Dense(
          latent_dims, use_bias=True, activation="relu", name="latent"),
  ], name="analysis_transform")

def decode():
  """Creates the synthesis (decoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Dense(
          img_length, use_bias=True, activation="sigmoid", name="outputs"),
  ], name="synthesis_transform")


def estimate_rate(x):
    def entropy(row):
        _, _, count = tf.unique_with_counts(row)
        prob = count / tf.reduce_sum(count)
        _2 = tf.ones_like(prob)*2
        log_2 = tf.cast(tf.math.log(_2), tf.float64)
        #print(log_2.dtype, prob.dtype, tf.math.log(prob).dtype)
        #return -tf.reduce_sum(prob * tf.math.log(prob))
        return -tf.reduce_sum(prob * tf.math.log(prob)/log_2)

    #     rev = tf.map_fn(row_entropy, new_f_w_t,dtype=tf.float32)
    #entropy_batch = []
    #for j in range(len(batch)):
    #    entropy_batch.append(entropy(batch, base=2))
    #new_f_w_t = tf.histogram_fixed_width_bins(x, value_ranges, nbins)
    #rev = tf.map_fn(row_entropy, new_f_w_t,dtype=tf.float32)
    value_ranges = [0.0, 32.0]
    nbins = 256
    #print("estimate_rate.max(x)=",
    #      tf.reduce_max(x, axis = 1, keepdims = True),
    #      "estimate_rate.min(x)=",
    #      tf.reduce_min(x, axis = 1, keepdims = True))
    histograms = tf.cast(tf.histogram_fixed_width_bins(x, value_ranges, nbins), tf.float32)
    entropies = tf.map_fn(fn=entropy, elems=histograms, dtype=tf.float64)
    rate = tf.reduce_sum(entropies)
    #print("----------------___", histograms.dtype, entropies.dtype, rate.dtype)
    
    rate /= (tf.cast(tf.size(entropies), tf.float64))
    #tf.print("rate=", rate)
    return rate

class Trainer(tf.keras.Model):

    def __init__(self, latent_dims):
        super().__init__()
        self.encode = encode(latent_dims)
        self.decode = decode()

    def call(self, x):
        """Computes rate and distortion losses."""
        # Ensure inputs are floats in the range (0, 1).
        #tf.print("x=", x) # <- produces 0
        #print("max x", tf.reduce_max(x))
        
        x = tf.cast(x, self.compute_dtype) / 255.
        x = tf.reshape(x, (-1, 28*28))

        # Compute latent space representation y, perturb it and model its entropy,
        # then compute the reconstructed pixel-level representation x_hat.
        y = self.encode(x)
        #print("size(y)=", tf.size(y))
        #print("len(y)=", len(y))
        #print("shape(y)", tf.shape(y))
        #print("size(x)=", tf.size(x))
        #tf.print("x=", x)
        #tf.print("y=", y)

        rate = estimate_rate(y)
        x_hat = self.decode(y)
        #print("Autoencoder.call.max(y)=", tf.reduce_max(y))
        #print("Autoencoder.call.max(x)=", tf.reduce_max(x))
        #print("Autoencoder.call.max(x_hat)=", tf.reduce_max(x_hat))

        # Average number of bits per MNIST digit.
        rate = tf.reduce_mean(rate)

        # Mean absolute difference across pixels.
        distortion = tf.reduce_mean(abs(x - x_hat))

        return dict(rate=rate, distortion=distortion)

(example_batch, _), = test_DS.batch(32).take(1)
trainer = Trainer(10)
example_output = trainer(example_batch)

print("rate: ", example_output["rate"])
print("distortion: ", example_output["distortion"])

def pass_through_loss(_, x):
  # Since rate and distortion are unsupervised, the loss doesn't need a target.
  return x

def configure_training(lmbda, latent_dims=32):
    trainer = Trainer(latent_dims)
    trainer.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        # Just pass through rate and distortion as losses/metrics.
        loss=dict(rate=pass_through_loss, distortion=pass_through_loss),
        metrics=dict(rate=pass_through_loss, distortion=pass_through_loss),
        loss_weights=dict(rate=1., distortion=lmbda),)
    return trainer

def add_rd_targets(image, label):
    # Training is unsupervised, so labels aren't necessary here. However, we
    # need to add "dummy" targets for rate and distortion.
    #tf.print(image) <- produce 0
    #tf.print(label) <- produce labels
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    #processed_img = tf.cast(processed_img, tf.float32) / 255.
    #tf.print(processed_img)
    return processed_img, dict(rate=0., distortion=0.)

def train(lmbda, epochs=5):
    trainer = configure_training(lmbda)
    #print("-----------")
    #tf.print(train_DS.take(16))
    #print("-----------")
    trainer.fit(
        train_DS.map(add_rd_targets).batch(128).prefetch(8),
        epochs=epochs,
        validation_data=test_DS.map(add_rd_targets).batch(128).cache(),
        #validation_data=train_DS.map(add_rd_targets).batch(128).cache(),
        validation_freq=1,
        verbose=1,
    )
    return trainer

trainer = train(lmbda=5000)
trainer.summary()

class Compressor(tf.keras.Model):
    def __init__(self, analysis_transform):
        super().__init__()
        self.analysis_transform = analysis_transform

    def call(self, x):
        # Ensure inputs are floats in the range (0, 1).
        x = tf.cast(x, self.compute_dtype) / 255.
        x = tf.reshape(x, (-1, 28*28))
        y = self.analysis_transform(x)
        #tf.print(y)
        #print("size(y)=", tf.size(y))
        # Also return the exact information content of each digit.
        rate = estimate_rate(y)
        rate = tf.reduce_mean(rate)
        return y, rate

class Decompressor(tf.keras.Model):
    def __init__(self, synthesis_transform):
        super().__init__()
        self.synthesis_transform = synthesis_transform

    def call(self, y):
        #tf.print(y)
        #print("Decompressor.call.max(y)", tf.reduce_max(y)) # Returns > 0
        x_hat = self.synthesis_transform(y)
        #print("Decompressor.call.max(x_hat)", tf.reduce_max(x_hat))
        # Scale and cast back to 8-bit integer.
        #tf.print(x_hat)
        #print("Decompressor.call.x_hat", tf.reduce_max(x_hat)) # Returns 0
        return tf.saturate_cast(tf.round(x_hat * 255.), tf.uint8)

def make_mnist_codec(trainer, **kwargs):
    # The entropy model must be created with `compression=True` and the same
    # instance must be shared between compressor and decompressor.
    compressor = Compressor(trainer.encode)
    decompressor = Decompressor(trainer.decode)
    return compressor, decompressor

compressor, decompressor = make_mnist_codec(trainer)

(originals, _), = test_DS.batch(16).skip(3).take(1)
y, entropies = compressor(originals)
print("shape(y)=", tf.shape(y), "max(y)=", tf.reduce_max(y), "entropies=", entropies)
reconstructed_imgs = decompressor(y)
print("shape(recons)=", tf.shape(reconstructed_imgs), "max(recons)=", tf.reduce_max(reconstructed_imgs))
n = len(reconstructed_imgs)
plt.gray()
plt.figure(figsize=(40,4))
for i in range(n):
    ax = plt.subplot(3, n, i + 1 )
    plt.imshow(tf.reshape(originals[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(resize(y[i],(LATENT_SPACE_LENGTH, img_width)))
    #plt.imshow(tf.reshape(reconstructed_imgs[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + 2*n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(tf.reshape(reconstructed_imgs[i], (img_height, img_width)))
    plt.axis("off")
plt.show()

##  5. Adding some convolutional layers
Now, the dense ANN learns on shapes, not on pixels.

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
#from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

train_DS, test_DS = tfds.load(
    DATASET,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)

def encode(latent_dims):
  """Creates the analysis (encoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2D(
          50, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          latent_dims, use_bias=True, activation="relu", name="latent"),
  ], name="analysis_transform")

def decode():
  """Creates the synthesis (decoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          2450, use_bias=True, activation="leaky_relu", name="fc_2"),
      tf.keras.layers.Reshape((7, 7, 50)),
      tf.keras.layers.Conv2DTranspose(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2DTranspose(
          1, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
  ], name="synthesis_transform")

def estimate_rate(x):
    def entropy(row):
        _, _, count = tf.unique_with_counts(row)
        prob = count / tf.reduce_sum(count)
        _2 = tf.ones_like(prob)*2
        log_2 = tf.cast(tf.math.log(_2), tf.float64)
        return -tf.reduce_sum(prob * tf.math.log(prob)/log_2)

    value_ranges = [0.0, 32.0]
    nbins = 256
    print("estimate_rate.max(x)=",
          tf.reduce_max(x, axis = 1, keepdims = True),
          "estimate_rate.min(x)=",
          tf.reduce_min(x, axis = 1, keepdims = True))
    histograms = tf.cast(tf.histogram_fixed_width_bins(x, value_ranges, nbins), tf.float32)
    entropies = tf.map_fn(fn=entropy, elems=histograms, dtype=tf.float64)
    rate = tf.reduce_sum(entropies)
    
    rate /= (tf.cast(tf.size(entropies), tf.float64))
    return rate

class Trainer(tf.keras.Model):

    def __init__(self, latent_dims):
        super().__init__()
        self.encode = encode(latent_dims)
        self.decode = decode()

    def call(self, x):
        """Computes rate and distortion losses."""
        # Ensure inputs are floats in the range (0, 1).
        x = tf.cast(x, self.compute_dtype) / 255.
        x = tf.reshape(x, (-1, 28, 28, 1))

        # Compute latent space representation y, perturb it and model its entropy,
        # then compute the reconstructed pixel-level representation x_hat.
        y = self.encode(x)

        rate = estimate_rate(y)
        x_hat = self.decode(y)

        # Average number of bits per MNIST digit.
        rate = tf.reduce_mean(rate)

        # Mean absolute difference across pixels.
        distortion = tf.reduce_mean(abs(x - x_hat))

        return dict(rate=rate, distortion=distortion)

(example_batch, _), = test_DS.batch(32).take(1)
trainer = Trainer(10)
example_output = trainer(example_batch)

print("rate: ", example_output["rate"])
print("distortion: ", example_output["distortion"])

def pass_through_loss(_, x):
  # Since rate and distortion are unsupervised, the loss doesn't need a target.
  return x

def configure_training(lmbda, latent_dims=32):
    trainer = Trainer(latent_dims)
    trainer.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        # Just pass through rate and distortion as losses/metrics.
        loss=dict(rate=pass_through_loss, distortion=pass_through_loss),
        metrics=dict(rate=pass_through_loss, distortion=pass_through_loss),
        loss_weights=dict(rate=1., distortion=lmbda),)
    return trainer

def add_rd_targets(image, label):
    # Training is unsupervised, so labels aren't necessary here. However, we
    # need to add "dummy" targets for rate and distortion.
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    #processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, dict(rate=0., distortion=0.)

def train(lmbda, epochs=5):
    trainer = configure_training(lmbda)
    trainer.fit(
        train_DS.map(add_rd_targets).batch(128).prefetch(8),
        epochs=epochs,
        validation_data=test_DS.map(add_rd_targets).batch(128).cache(),
        validation_freq=1,
        verbose=1,
    )
    return trainer

trainer = train(lmbda=500)
trainer.summary()

class Compressor(tf.keras.Model):
    def __init__(self, analysis_transform):
        super().__init__()
        self.analysis_transform = analysis_transform

    def call(self, x):
        # Ensure inputs are floats in the range (0, 1).
        x = tf.cast(x, self.compute_dtype) / 255.
        #x = tf.reshape(x, (-1, 28*28))
        y = self.analysis_transform(x)
        # Also return the exact information content of each digit.
        rate = estimate_rate(y)
        rate = tf.reduce_mean(rate)
        return y, rate

class Decompressor(tf.keras.Model):
    def __init__(self, synthesis_transform):
        super().__init__()
        self.synthesis_transform = synthesis_transform

    def call(self, y):
        x_hat = self.synthesis_transform(y)
        # Scale and cast back to 8-bit integer.
        return tf.saturate_cast(tf.round(x_hat * 255.), tf.uint8)

def make_mnist_codec(trainer, **kwargs):
    # The entropy model must be created with `compression=True` and the same
    # instance must be shared between compressor and decompressor.
    compressor = Compressor(trainer.encode)
    decompressor = Decompressor(trainer.decode)
    return compressor, decompressor

compressor, decompressor = make_mnist_codec(trainer)
(originals, _), = test_DS.batch(16).skip(3).take(1)
y, entropies = compressor(originals)
print("shape(y)=", tf.shape(y), "max(y)=", tf.reduce_max(y), "entropies=", entropies)
reconstructed_imgs = decompressor(y)
print("shape(recons)=", tf.shape(reconstructed_imgs), "max(recons)=", tf.reduce_max(reconstructed_imgs))
n = len(reconstructed_imgs)
plt.gray()
plt.figure(figsize=(40,4))
for i in range(n):
    ax = plt.subplot(3, n, i + 1 )
    plt.imshow(tf.reshape(originals[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(resize(y[i],(LATENT_SPACE_LENGTH, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + 2*n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(tf.reshape(reconstructed_imgs[i], (img_height, img_width)))
    plt.axis("off")
plt.show()

In [ ]:
import os

print(tf.shape(y))

#latent = tf.constant([os.urandom(8) for _ in range(16)])
#tf.print(latent)
latent = tf.random.stateless_normal(shape=[16, LATENT_SPACE_LENGTH], seed=[1, 2])
print(latent)
#input()
samples = decompressor(latent)

fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(5, 5))
axes = axes.ravel()
for i in range(len(axes)):
  axes[i].imshow(tf.squeeze(samples[i]))
  axes[i].axis("off")
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)


##  6. Variational Autoencoding
A VAE is an autoencoder that learns a latent variable model for its input data. So instead of letting the neural network learn an arbitrary function, it is learning the parameters of a probability distribution modeling the data. Now, if you sample points from this distribution, you can generate new input data samples: a VAE is a "generative model". See https://blog.keras.io/building-autoencoders-in-keras.html.

In [ ]:
# Based on https://keras.io/examples/generative/vae/ and 
# https://www.tensorflow.org/tutorials/generative/data_compression

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

EPOCHS = 1
#EPOCHS = 30
LAMBDA = 1.0

def estimate_rate(x):
    def entropy(row):
        _, _, count = tf.unique_with_counts(row)
        prob = count / tf.reduce_sum(count)
        _2 = tf.ones_like(prob)*2
        log_2 = tf.cast(tf.math.log(_2), tf.float64)
        return -tf.reduce_sum(prob * tf.math.log(prob)/log_2)

    value_ranges = [0.0, 32.0]
    nbins = 256
    print("estimate_rate.max(x)=",
          tf.reduce_max(x, axis = 1, keepdims = True),
          "estimate_rate.min(x)=",
          tf.reduce_min(x, axis = 1, keepdims = True))
    histograms = tf.cast(tf.histogram_fixed_width_bins(x, value_ranges, nbins), tf.float32)
    entropies = tf.map_fn(fn=entropy, elems=histograms, dtype=tf.float64)
    rate = tf.reduce_sum(entropies)
    
    rate /= (tf.cast(tf.size(entropies), tf.float64))
    return tf.cast(rate, tf.float32)

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            z_mean_entropy = estimate_rate(z_mean)
            z_log_var_entropy = estimate_rate(z_log_var)
            total_entropy = (z_mean_entropy + z_log_var_entropy)/2.0  # Both have the same length
            tf.print("\ntotal_entropy=", total_entropy)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = total_entropy + LAMBDA*(reconstruction_loss + kl_loss)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x = np.concatenate([x_train, x_test], axis=0)
x = np.expand_dims(x, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(x=x, epochs=EPOCHS, batch_size=128)

In [ ]:
import matplotlib.pyplot as plt

def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)


In [ ]:
z_mean, z_log_var, z = encoder.predict(x[0:16])
y = decoder.predict(z)

In [ ]:
n = len(y)
plt.gray()
plt.figure(figsize=(40,4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1 )
    plt.imshow(tf.reshape(x[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(2, n, i + 1 + n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(tf.reshape(y[i], (img_height, img_width)))
    plt.axis("off")
plt.show()

In [ ]:
# https://keras.io/examples/generative/vae/

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#EPOCHS = 5
EPOCHS = 20

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=EPOCHS, batch_size=128)

import matplotlib.pyplot as plt

def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)


In [ ]:
z_mean, z_log_var, z = encoder.predict(x[0:16])
y = decoder.predict(z)
print("sampling (one for each encoded image):")
print(tf.shape(z))
tf.print(z)
print("means:")
print(tf.shape(z_mean))
tf.print(z_mean)
print("log variances:")
print(tf.shape(z_log_var))
tf.print(z_log_var)
n = len(y)
plt.gray()
plt.figure(figsize=(40,4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1 )
    plt.imshow(tf.reshape(x[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(2, n, i + 1 + n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(tf.reshape(y[i], (img_height, img_width)))
    plt.axis("off")
plt.show()

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
#from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

train_DS, test_DS = tfds.load(
    DATASET,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)

def encode(latent_dims):
  """Creates the analysis (encoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2D(
          50, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          latent_dims, use_bias=True, activation="relu", name="latent"),
  ], name="analysis_transform")

def decode():
  """Creates the synthesis (decoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          2450, use_bias=True, activation="leaky_relu", name="fc_2"),
      tf.keras.layers.Reshape((7, 7, 50)),
      tf.keras.layers.Conv2DTranspose(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2DTranspose(
          1, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
  ], name="synthesis_transform")

def estimate_rate(x):
    def entropy(row):
        _, _, count = tf.unique_with_counts(row)
        prob = count / tf.reduce_sum(count)
        _2 = tf.ones_like(prob)*2
        log_2 = tf.cast(tf.math.log(_2), tf.float64)
        return -tf.reduce_sum(prob * tf.math.log(prob)/log_2)

    value_ranges = [0.0, 32.0]
    nbins = 256
    print("estimate_rate.max(x)=",
          tf.reduce_max(x, axis = 1, keepdims = True),
          "estimate_rate.min(x)=",
          tf.reduce_min(x, axis = 1, keepdims = True))
    histograms = tf.cast(tf.histogram_fixed_width_bins(x, value_ranges, nbins), tf.float32)
    entropies = tf.map_fn(fn=entropy, elems=histograms, dtype=tf.float64)
    rate = tf.reduce_sum(entropies)
    
    rate /= (tf.cast(tf.size(entropies), tf.float64))
    return rate

class Trainer(tf.keras.Model):

    def __init__(self, latent_dims):
        super().__init__()
        self.encode = encode(latent_dims)
        self.decode = decode()

    def call(self, x):
        """Computes rate and distortion losses."""
        # Ensure inputs are floats in the range (0, 1).
        x = tf.cast(x, self.compute_dtype) / 255.
        x = tf.reshape(x, (-1, 28, 28, 1))

        # Compute latent space representation y, perturb it and model its entropy,
        # then compute the reconstructed pixel-level representation x_hat.
        y = self.encode(x)

        rate = estimate_rate(y)
        x_hat = self.decode(y)

        # Average number of bits per MNIST digit.
        rate = tf.reduce_mean(rate)

        # Mean absolute difference across pixels.
        distortion = tf.reduce_mean(abs(x - x_hat))

        return dict(rate=rate, distortion=distortion)

(example_batch, _), = test_DS.batch(32).take(1)
trainer = Trainer(10)
example_output = trainer(example_batch)

print("rate: ", example_output["rate"])
print("distortion: ", example_output["distortion"])

def pass_through_loss(_, x):
  # Since rate and distortion are unsupervised, the loss doesn't need a target.
  return x

def configure_training(lmbda, latent_dims=50):
    trainer = Trainer(latent_dims)
    trainer.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        # Just pass through rate and distortion as losses/metrics.
        loss=dict(rate=pass_through_loss, distortion=pass_through_loss),
        metrics=dict(rate=pass_through_loss, distortion=pass_through_loss),
        loss_weights=dict(rate=1., distortion=lmbda),)
    return trainer

def add_rd_targets(image, label):
    # Training is unsupervised, so labels aren't necessary here. However, we
    # need to add "dummy" targets for rate and distortion.
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    #processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, dict(rate=0., distortion=0.)

def train(lmbda, epochs=5):
    trainer = configure_training(lmbda)
    trainer.fit(
        train_DS.map(add_rd_targets).batch(128).prefetch(8),
        epochs=epochs,
        validation_data=test_DS.map(add_rd_targets).batch(128).cache(),
        validation_freq=1,
        verbose=1,
    )
    return trainer

trainer = train(lmbda=5000)

class Compressor(tf.keras.Model):
    def __init__(self, analysis_transform):
        super().__init__()
        self.analysis_transform = analysis_transform

    def call(self, x):
        # Ensure inputs are floats in the range (0, 1).
        x = tf.cast(x, self.compute_dtype) / 255.
        #x = tf.reshape(x, (-1, 28*28))
        y = self.analysis_transform(x)
        # Also return the exact information content of each digit.
        rate = estimate_rate(y)
        rate = tf.reduce_mean(rate)
        return y, rate

class Decompressor(tf.keras.Model):
    def __init__(self, synthesis_transform):
        super().__init__()
        self.synthesis_transform = synthesis_transform

    def call(self, y):
        x_hat = self.synthesis_transform(y)
        # Scale and cast back to 8-bit integer.
        return tf.saturate_cast(tf.round(x_hat * 255.), tf.uint8)

def make_mnist_codec(trainer, **kwargs):
    # The entropy model must be created with `compression=True` and the same
    # instance must be shared between compressor and decompressor.
    compressor = Compressor(trainer.encode)
    decompressor = Decompressor(trainer.decode)
    return compressor, decompressor

compressor, decompressor = make_mnist_codec(trainer)
(originals, _), = test_DS.batch(16).skip(3).take(1)
y, entropies = compressor(originals)
print("shape(y)=", tf.shape(y), "max(y)=", tf.reduce_max(y), "entropies=", entropies)
reconstructed_imgs = decompressor(y)
print("shape(recons)=", tf.shape(reconstructed_imgs), "max(recons)=", tf.reduce_max(reconstructed_imgs))
n = len(reconstructed_imgs)
plt.gray()
plt.figure(figsize=(40,4))
for i in range(n):
    ax = plt.subplot(3, n, i + 1 )
    plt.imshow(tf.reshape(originals[i], (img_height, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(resize(y[i],(LATENT_SPACE_LENGTH, img_width)))
    plt.axis("off")
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + 2*n)
    #tf.print(reconstructed_imgs[i])
    plt.imshow(tf.reshape(reconstructed_imgs[i], (img_height, img_width)))
    plt.axis("off")
plt.show()

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['x_train']
    x_test = data['x_test']
    y_test = data['x_test']
    
# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_DS = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Define how to use the datasets, preprocess, etc.

def process_x(image, label):
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, label

def process_y(image, label):
    # The last layer of the network is a dense layer and TensorFlow imposes 1D.
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(process_x)
train_DS = train_DS.map(process_y)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(process_x)
test_DS = test_DS.map(process_y)
test_DS = test_DS.batch(BATCH_SIZE)

class NoisyDense(keras.layers.Layer):

    def __init__(self, units=32, input_dim=32, activation="relu", **kwargs):
        super(NoisyDense, self).__init__(**kwargs)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )
        self.quantized_vals = 0

    #@tf.function
    def call(self, inputs):
        outputs = tf.matmul(inputs, self.w) + self.b
        #noise = tf.random.uniform(shape=tf.shape(weighted_sums), minval=-.5, maxval=.5)
        #quantized_sums_01 = weighted_sums
        #quantized_sums_01 = noise + weighted_sums
        #print(tf.reduce_max(weighted_sums))
        #outputs = tf.keras.activations.tanh(outputs)
        outputs = tf.saturate_cast(tf.round(outputs * 255.), tf.int16)
        #quantized_sums_8b = tf.saturate_cast(tf.round(weighted_sums * 255.), tf.int16)
        self.quantized_vals = outputs
        #print(len(self.quantized_vals))
        outputs = tf.cast(outputs, tf.float32) / 255.0
        #activations = tf.keras.activations.relu(quantized_sums_01)
        #activations = quantized_sums_01
        #print(len(activations))
        #tf.print(activations)
        #print(type(activations))
        #print("act[0][0] =",activations[0][0])
        #print(activations.numpy())
        #activations_entropy = entropy(tf.make_ndarray(activations), base=2)
        #self.codestream_length = activations_entropy * tf.size(activations)
        return outputs
    
    def get_vals(self):
        return self.quantized_vals

# Define the autoencoder.
input_layer = keras.Input(
    shape=(img_length,),
    name="inputs")
hidden_layer = NoisyDense(
    units=LATENT_SPACE_LENGTH,
    input_dim = img_length,
    activation="relu",
    name="hidden"
    )(input_layer)
output_layer = layers.Dense(
    img_length,
    activation="sigmoid",
    name="outputs")(hidden_layer)

autoencoder = keras.Model(
    input_layer,
    output_layer)

# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(
    input_layer,
    hidden_layer)

autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy", run_eagerly=True
)

autoencoder.summary()

# Train.
EPOCHS = 1
ITERATIONS = 1
plt.gray()
for i in range(ITERATIONS):

    autoencoder.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    batch_vals = autoencoder.get_layer("hidden").get_vals()
    print("batch_vals =", batch_vals)
    for j in range(BATCH_SIZE):
        vals = batch_vals[j].numpy() + 512
        #print(vals)
        print(entropy(vals, base=2))
    
    #print("2", autoencoder.get_layer("hidden").variables[1])

    # Show the learning.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    # Show the originals (using the pipeline).
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(3, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                #plt.gray()
                plt.axis("off")
            except:
                pass
    
    # See the latent space (as images).
    latent_imgs = encoder.predict(test_DS)
    print("type latent", type(latent_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
        plt.axis("off")    

    # See the reconstructions.
    reconstructed_imgs = autoencoder.predict(test_DS)
    print(type(reconstructed_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.axis("off")    

    plt.show()

##  Quantizing only during the inference
During the training we add uniform noise between $[-0.5, 0.5]$ to the latent space in order to simulate the distortion generated by the quantization during the generation of the predictions (see [End-to-end Optimized Image Compression](https://arxiv.org/abs/1611.01704)).

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from scipy.stats import entropy
#tf.compat.v1.enable_eager_execution()

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_SPACE_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['x_train']
    x_test = data['x_test']
    y_test = data['x_test']
    
# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_DS = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Define how to use the datasets, preprocess, etc.

def process_x(image, label):
    processed_img = tf.reshape(image, (img_height * img_width, 1))
    processed_img = tf.cast(processed_img, tf.float32) / 255.
    return processed_img, label

def process_y(image, label):
    # The last layer of the network is a dense layer and TensorFlow imposes 1D.
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(process_x)
train_DS = train_DS.map(process_y)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(process_x)
test_DS = test_DS.map(process_y)
test_DS = test_DS.batch(BATCH_SIZE)

class NoisyDense(keras.layers.Layer):

    def __init__(self, units=32, input_dim=32, activation="relu", **kwargs):
        super(NoisyDense, self).__init__(**kwargs)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )
        self.quantized_vals = 0

    #@tf.function
    def call(self, inputs):
        outputs = tf.matmul(inputs, self.w) + self.b
        noise = tf.random.uniform(shape=tf.shape(outputs), minval=-.5, maxval=.5)
        #quantized_sums_01 = weighted_sums
        outputs += noise
        #print(tf.reduce_max(weighted_sums))
        #outputs = tf.keras.activations.tanh(outputs)
        #outputs = tf.saturate_cast(tf.round(outputs * 255.), tf.int16)
        #quantized_sums_8b = tf.saturate_cast(tf.round(weighted_sums * 255.), tf.int16)
        self.quantized_vals = outputs
        #print(len(self.quantized_vals))
        #outputs = tf.cast(outputs, tf.float32) / 255.0
        #activations = tf.keras.activations.relu(quantized_sums_01)
        #activations = quantized_sums_01
        #print(len(activations))
        #tf.print(activations)
        #print(type(activations))
        #print("act[0][0] =",activations[0][0])
        #print(activations.numpy())
        #activations_entropy = entropy(tf.make_ndarray(activations), base=2)
        #self.codestream_length = activations_entropy * tf.size(activations)
        return outputs
    
    def get_vals(self):
        return self.quantized_vals

# Define the autoencoder.
input_layer = keras.Input(
    shape=(img_length,),
    name="inputs")
hidden_layer = NoisyDense(
    units=LATENT_SPACE_LENGTH,
    input_dim = img_length,
    activation="relu",
    name="hidden"
    )(input_layer)
output_layer = layers.Dense(
    img_length,
    activation="sigmoid",
    name="outputs")(hidden_layer)

autoencoder = keras.Model(
    input_layer,
    output_layer)

# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(
    input_layer,
    hidden_layer)

autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy", run_eagerly=True
)

autoencoder.summary()

# Train.
EPOCHS = 1
ITERATIONS = 1
plt.gray()
for i in range(ITERATIONS):

    autoencoder.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    batch_vals = autoencoder.get_layer("hidden").get_vals()
    print("batch_vals =", batch_vals)
    for j in range(BATCH_SIZE):
        try:
            vals = batch_vals[j].numpy() + 512
            print(entropy(vals, base=2))
        except:
            pass
        #print(vals)
    
    #print("2", autoencoder.get_layer("hidden").variables[1])

    # Show the learning.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    # Show the originals (using the pipeline).
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(3, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                #plt.gray()
                plt.axis("off")
            except:
                pass
    
    # See the latent space (as images).
    latent_imgs = encoder.predict(test_DS)
    print("type latent", type(latent_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
        plt.axis("off")    
    
    # See the reconstructions.
    reconstructed_imgs = autoencoder.predict(test_DS)
    print(type(reconstructed_imgs))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.axis("off")    

    plt.show()

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import tensorflow_compression as tfc

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

LATENT_LENGTH = 32

img_width = 28
img_height = 28
img_length = img_height * img_width

training_DS, test_DS = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)

# Define the autoencoder.
def analyze(latent_length):
  return tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2D(
          50, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(
          img_length, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          latent_length, use_bias=True, activation=None, name="fc_2"),
  ], name="analysis_transform")

def synthesize():
  return tf.keras.Sequential([
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          2450, use_bias=True, activation="leaky_relu", name="fc_2"),
      tf.keras.layers.Reshape((7, 7, 50)),
      tf.keras.layers.Conv2DTranspose(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2DTranspose(
          1, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
  ], name="synthesis_transform")

class MNISTCompressionTrainer(tf.keras.Model):
  """Model that trains a compressor/decompressor for MNIST."""

  def __init__(self, latent_dims):
    super().__init__()
    self.analysis_transform = analyze(latent_dims)
    self.synthesis_transform = synthesize()
    self.prior_log_scales = tf.Variable(tf.zeros((latent_dims,)))

  @property
  def prior(self):
    return tfc.NoisyLogistic(loc=0., scale=tf.exp(self.prior_log_scales))

  def call(self, x, training):
    """Computes rate and distortion losses."""
    # Ensure inputs are floats in the range (0, 1).
    x = tf.cast(x, self.compute_dtype) / 255.
    x = tf.reshape(x, (-1, 28, 28, 1))

    # Compute latent space representation y, perturb it and model its entropy,
    # then compute the reconstructed pixel-level representation x_hat.
    y = self.analysis_transform(x)
    entropy_model = tfc.ContinuousBatchedEntropyModel(
        self.prior, coding_rank=1, compression=False)
    y_tilde, rate = entropy_model(y, training=training)
    x_tilde = self.synthesis_transform(y_tilde)

    # Average number of bits per MNIST digit.
    rate = tf.reduce_mean(rate)

    # Mean absolute difference across pixels.
    distortion = tf.reduce_mean(abs(x - x_tilde))

    return dict(rate=rate, distortion=distortion)

def pass_through_loss(_, x):
  # Since rate and distortion are unsupervised, the loss doesn't need a target.
  return x

def make_mnist_compression_trainer(lmbda, latent_dims=50):
  trainer = MNISTCompressionTrainer(latent_dims)
  trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    # Just pass through rate and distortion as losses/metrics.
    loss=dict(rate=pass_through_loss, distortion=pass_through_loss),
    metrics=dict(rate=pass_through_loss, distortion=pass_through_loss),
    loss_weights=dict(rate=1., distortion=lmbda),
  )
  return trainer

def add_rd_targets(image, label):
  # Training is unsupervised, so labels aren't necessary here. However, we
  # need to add "dummy" targets for rate and distortion.
  return image, dict(rate=0., distortion=0.)

def train_mnist_model(lmbda):
  trainer = make_mnist_compression_trainer(lmbda)
  trainer.fit(
      training_DS.map(add_rd_targets).batch(128).prefetch(8),
      epochs=1,
      validation_data=test_DS.map(add_rd_targets).batch(128).cache(),
      validation_freq=1,
      verbose=1,
  )
  return trainer

trainer = train_mnist_model(lmbda=2000)
trainer.summary()

class MNISTCompressor(tf.keras.Model):
  """Compresses MNIST images to strings."""

  def __init__(self, analysis_transform, entropy_model):
    super().__init__()
    self.analysis_transform = analysis_transform
    self.entropy_model = entropy_model

  def call(self, x):
    # Ensure inputs are floats in the range (0, 1).
    x = tf.cast(x, self.compute_dtype) / 255.
    y = self.analysis_transform(x)
    # Also return the exact information content of each digit.
    _, bits = self.entropy_model(y, training=False)
    return self.entropy_model.compress(y), bits

class MNISTDecompressor(tf.keras.Model):
  """Decompresses MNIST images from strings."""

  def __init__(self, entropy_model, synthesis_transform):
    super().__init__()
    self.entropy_model = entropy_model
    self.synthesis_transform = synthesis_transform

  def call(self, string):
    y_hat = self.entropy_model.decompress(string, ())
    x_hat = self.synthesis_transform(y_hat)
    # Scale and cast back to 8-bit integer.
    return tf.saturate_cast(tf.round(x_hat * 255.), tf.uint8)

def make_mnist_codec(trainer, **kwargs):
  # The entropy model must be created with `compression=True` and the same
  # instance must be shared between compressor and decompressor.
  entropy_model = tfc.ContinuousBatchedEntropyModel(
      trainer.prior, coding_rank=1, compression=True, **kwargs)
  compressor = MNISTCompressor(trainer.analysis_transform, entropy_model)
  decompressor = MNISTDecompressor(entropy_model, trainer.synthesis_transform)
  return compressor, decompressor

compressor, decompressor = make_mnist_codec(trainer)

(originals, _), = test_DS.batch(16).skip(3).take(1)
strings, entropies = compressor(originals)

print(f"String representation of first digit in hexadecimal: 0x{strings[0].numpy().hex()}")
print(f"Number of bits actually needed to represent it: {entropies[0]:0.2f}")
reconstructions = decompressor(strings)
print(type(reconstructions))


def display_digits(originals, strings, entropies, reconstructions):
  """Visualizes 16 digits together with their reconstructions."""
  fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(12.5, 5))
  axes = axes.ravel()
  for i in range(len(axes)):
    image = tf.concat([
        tf.squeeze(originals[i]),
        tf.zeros((28, 14), tf.uint8),
        tf.squeeze(reconstructions[i]),
    ], 1)
    axes[i].imshow(image)
    axes[i].text(
        .5, .5, f"→ 0x{strings[i].numpy().hex()} →\n{entropies[i]:0.2f} bits",
        ha="center", va="top", color="white", fontsize="small",
        transform=axes[i].transAxes)
    axes[i].axis("off")
  plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)

#display_digits(originals, strings, entropies, reconstructions)

#input()

# Show the learning.
plt.figure(figsize=(40,4))

n = len(originals)
if n > 32:
    n = 32

# Show the originals (using the pipeline).
for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(originals[i])
    plt.axis("off")    

# See the latent space (as images).
#latent_imgs = encoder.predict(test_DS)
#for i in range(n):
#    ax = plt.subplot(3, n, i + 1 + n)
#    plt.imshow(resize(latent_imgs[i],(LATENT_SPACE_LENGTH, img_width)))
#    plt.axis("off")    

# See the reconstructions.
for i in range(n):
    ax = plt.subplot(3, n, i + 1 + 1*n)
    plt.imshow(reconstructions[i])
    plt.axis("off")    

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_compression as tfc
import tensorflow_datasets as tfds

In [ ]:
while True:
    time.sleep(1)

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize

LATENT_SPACE_LENGTH = 32

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'mnist'
#DATASET = 'fashion_mnist'
#URL = "https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz"

img_width = 28
img_height = 28
img_length = img_height * img_width

# Generate the NumPy arrays with the data.
DATA_URL = f'https://storage.googleapis.com/tensorflow/tf-keras-datasets/{DATASET}.npz'
path = tf.keras.utils.get_file(f'{DATASET}.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

print(type(train_examples), type(train_labels))
print(len(train_examples), len(test_labels))

# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_DS = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

print(type(train_DS))

# Define how to use the datasets, preprocess, etc.

def normalize_img(image, label):
    processed_img = tf.cast(image, tf.float32) / 255.
    print(processed_img.shape)
    return processed_img, label

def get_image(image, label):
    return image, tf.reshape(image, (img_height * img_width, 1))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(normalize_img)
train_DS = train_DS.map(get_image)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(normalize_img)
test_DS = test_DS.map(get_image)
test_DS = test_DS.batch(BATCH_SIZE)

# Define the autoencoder.
img_pixels = img_height * img_width
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width)),
  tf.keras.layers.Dense(LATENT_SPACE_LENGTH, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

# Train.
EPOCHS = 1
ITERATIONS = 5
for i in range(ITERATIONS):

    model.fit(
        train_DS,
        epochs=EPOCHS,
        validation_data=test_DS)

    # Predict.
    reconstructed_imgs = model.predict(test_DS)

    # Show the predictions.
    plt.figure(figsize=(40,4))

    n = len(train_DS)
    if n > 32:
        n = 32

    ## original
    for images in test_DS.take(1):
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass
            
    # latent (as an image)
    ''' 
    for i in range(n):
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(resize(latents[i],(HIDDEN_SIZE, img_width)))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    '''

    ## reconstruction
    for i in range(n):
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.gray()
        plt.axis("off")

    plt.show()

## Using 3 hidden layer

    Input -> hidden_0 -> latent -> hidden_1 -> Output
    28x28    10x10       32        10x10       28x28

## Ignore the rest

In [ ]:
# Autoencoding example (feeding pipeline (tf.data.Dataset) with NumPy arrays)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
from skimage.transform import resize

# Generate the NumPy arrays with the data.
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

print(type(train_examples), type(train_labels))
print(len(train_examples), len(test_labels))

# Generate the pipelines.
train_DS = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_DS = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

print(type(train_DS))

# Define how to use the datasets, preprocess, etc.

def normalize_img(image, label):
    processed_img = tf.cast(image, tf.float32) / 255.
    print(processed_img.shape)
    return processed_img, label

def get_image(image, label):
    return image, tf.reshape(image, (28*28, 1))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_DS = train_DS.map(normalize_img)
train_DS = train_DS.map(get_image)
train_DS = train_DS.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_DS = test_DS.map(normalize_img)
test_DS = test_DS.map(get_image)
test_DS = test_DS.batch(BATCH_SIZE)

# Define the autoencoder.
img_pixels = 28*28
HIDDEN_SIZE = 32
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

# Train.
EPOCHS = 10
model.fit(
    train_DS,
    epochs=EPOCHS,
    validation_data=test_DS)

# Predict.
reconstructed_imgs = model.predict(test_DS)

# Show the predictions.
img_width = 28
img_height = 28
plt.figure(figsize=(40,4))

n = len(train_DS)
if n > 32:
    n = 32
        
## original
for images in test_DS.take(1):
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        try:
            image = images[0][i].numpy()
            image *= 255
            image = image.astype("uint8").reshape(img_height, img_width)
            plt.imshow(image)
            plt.gray()
            plt.axis("off")
        except:
            pass

## reconstruction
for i in range(n):
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
    plt.gray()
    plt.axis("off")

plt.show()

In [ ]:
!pip install -q tensorflow-datasets
#!pip install tfds-nightly

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64


# Configure the size of the hidden layer.
HIDDEN_SIZE = 32

In [ ]:
# Load the datasets (notice that we will use x_train = y_train and x_test = y_test)
(x_train, _), (x_test, _) = mnist.load_data()
print(x_train.shape, x_test.shape, type(x_train))

In [ ]:
# Gray-scale images ranges betwen [0, 255]. Neurons must work with values between [0, 1]
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

In [ ]:
img_height = x_train[0].shape[0]
img_width = x_train[0].shape[1]
img_pixels = img_width*img_height
print("The shape of the images is:", img_height, "x", img_width)

In [ ]:
# Our layers will be 1D. The images must be reshaped.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape, x_test.shape)

In [ ]:
# Define the network (2 layers), connecting them sequentially:
# flatten(input_image, 28*28) -> hidden_layer(HIDDEN_SIZE) -> output_layer(28*28)
input_layer = keras.Input(shape=(img_pixels,))
hidden_layer = layers.Dense(HIDDEN_SIZE, activation="relu")(input_layer)
output_layer = layers.Dense(img_pixels, activation="sigmoid")(hidden_layer)
autoencoder = keras.Model(input_layer, output_layer)

In [ ]:
# Compile the network.
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
autoencoder.summary()

In [ ]:
# Train the network.
autoencoder.fit(x=x_train, y=x_train,
                epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Apply the autoencoder to the test dataset.
reconstructed_imgs = autoencoder.predict(x_test)

In [ ]:
# Show some reconstructions.
n = 10
plt.figure(figsize=(40,4))
for i in range(n):
  # original
  ax = plt.subplot(2, n, i+1)
  plt.imshow(x_test[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  
  # reconstruction
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# To see the latents (content of the hidden layer), we define a new "model" with only the encoder.
encoder = keras.Model(input_layer, hidden_layer)

In [ ]:
latents = encoder.predict(x_test)

In [ ]:
from skimage.transform import resize
n = 10
plt.figure(figsize=(40,4))
for i in range(n):
  # original
  ax = plt.subplot(2, n, i+1)
  plt.imshow(x_test[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  
  # latent (as an image)
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(resize(latents[i],(HIDDEN_SIZE, img_width)))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

#### Images of any shape and the training dataset does not fit in memory

In [ ]:
!pip install -q tensorflow-datasets
#!pip install tfds-nightly

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

In [ ]:
# Show available datasets in TensorFlow.
#tfds.list_builders()

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64 # -1 -> All the dataset in memory!

# https://www.tensorflow.org/datasets/catalog/
DATASET = 'clic'  # 7.48 GiB :-()

In [ ]:
train_DS = tfds.load(
    name=DATASET,
    split='train',
    batch_size=BATCH_SIZE,
    shuffle_files=True) # During training, it's important to shuffle the data well - poorly shuffled data can result in lower training accuracy (https://www.tensorflow.org/datasets/performances).

test_DS = tfds.load(
    name=DATASET,
    split="test",
    batch_size=BATCH_SIZE)

In [ ]:
#....

In [ ]:
train_DS

In [ ]:
# Clustering example

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

print(ds_train, len(ds_train))

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  #print(image)
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
print(ds_train, len(ds_train))

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(28*28)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

reconstructed_imgs = model.predict(ds_test)
# Show some reconstructions.
n = 10

img_width = 28
img_height = 28

plt.figure(figsize=(40,4))
for i in range(n):
  # original
  ax = plt.subplot(2, n, i+1)
  #plt.imshow(np.squeeze(ds_test)[i].reshape(img_height, img_width))
  plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  
  # reconstruction
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Autoencoding example (using a data pipeline)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True, # Images are labeled
    with_info=True
)
print(ds_train, len(ds_train))

def normalize_img2(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    #print(image['image'].shape)
    #print(image)
    #return image, label
    return tf.cast(image, tf.float32) / 255., label

def get_image(image, label):
    return image, tf.reshape(image, (28*28, 1))

ds_train = ds_train.map(
    normalize_img2, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(get_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
#ds_train = ds_train.map(lambda x: x[0])

print(ds_train, len(ds_train))

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(get_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
#ds_test = ds_test.map(lambda x: x[0], ds_test)

print(ds_test, len(ds_test))

#x_test = list(map(lambda x: x[0], ds_test))
# = ds_test["image"]

img_pixels = 28*28

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_test)

reconstructed_imgs = model.predict(ds_test)

n = 10
plt.figure(figsize=(40,4))

# original
for images in ds_test.take(1):
    print(len(images))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        image = images[0][i].numpy()
        image *= 255
        image = image.astype("uint8").reshape(28, 28)
        plt.imshow(image)
        plt.gray()
        plt.axis("off")
        
# reconstruction
for i in range(n):
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
# Autoencoding example (iterative)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

img_pixels = 28*28
HIDDEN_SIZE = 32
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  #print(image)
  return tf.cast(image, tf.float32) / 255., label

def get_image(image, label):
    return image, tf.reshape(image, (28*28, 1))

img_width = 28
img_height = 28
ITERATIONS = 20
for i in range(ITERATIONS):
    
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True, # Images are labeled
        with_info=True)
    
    ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.map(get_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(
        ds_info.splits['train'].num_examples,
        reshuffle_each_iteration=True)
    ds_train = ds_train.batch(32)
    ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

    ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.map(get_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(32)
    ds_test = ds_test.cache()
    #ds_test = ds_test.shuffle(ds_info.splits['test[:10%]'].num_examples,
    #    reshuffle_each_iteration=True)
    ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

    print("len(train_DS) =", len(ds_train))
    print("len(test_DS) =", len(ds_test))
    
    n = len(ds_train)
    if n > 10:
        n = 10
    for images in ds_train.take(1):
        print(len(images))
        for i in range(min(len(ds_train), n)):
            ax = plt.subplot(2, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    EPOCHS = 1
    model.fit(
        ds_train,
        epochs=EPOCHS,
        validation_data=ds_test)

    reconstructed_imgs = model.predict(ds_test)

    plt.figure(figsize=(40,4))
    #plt.figure()

    # original
    for images in ds_test.take(1):
        #print(len(images))
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    # reconstruction
    for i in range(n):
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.gray()
        #ax.get_xaxis().set_visible(False)
        #ax.get_yaxis().set_visible(False)
        plt.axis("off")

    plt.show()

In [ ]:
# Autoencoding example (iterative, minimal)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

img_pixels = 28*28
HIDDEN_SIZE = 32
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  #print(image)
  return tf.cast(image, tf.float32) / 255., label

def get_image(image, label):
    return image, tf.reshape(image, (28*28, 1))
    
img_width = 28
img_height = 28
ITERATIONS = 20
for i in range(ITERATIONS):
    
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train[:1000]', 'test'],
        shuffle_files=True,
        as_supervised=True, # Images are labeled
        with_info=True)
    
    ds_train = ds_train.map(normalize_img)
    ds_train = ds_train.map(get_image)
    ds_train = ds_train.cache() # Do not put me in the last position
    ds_train = ds_train.shuffle(
        ds_info.splits['train'].num_examples,
        reshuffle_each_iteration=True)
    ds_train = ds_train.batch(32)

    ds_test = ds_test.map(normalize_img)
    ds_test = ds_test.map(get_image)
    ds_test = ds_test.shuffle(
        ds_info.splits['test'].num_examples,
        reshuffle_each_iteration=True)
    ds_test = ds_test.batch(32)
    ds_test = ds_test.cache() # Put me in the last position

    print("len(train_DS) =", len(ds_train))
    print("len(test_DS) =", len(ds_test))
    
    n = len(ds_train)
    if n > 32:
        n = 32
        
    plt.figure(figsize=(40,2))
    
    for images in ds_train.take(1):
        print(len(images))
        for i in range(min(len(ds_train), n)):
            ax = plt.subplot(1, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    EPOCHS = 1
    model.fit(
        ds_train,
        epochs=EPOCHS,
        validation_data=ds_test)

    reconstructed_imgs = model.predict(ds_test)

    plt.figure(figsize=(40,4))

    # original
    for images in ds_test.take(1):
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    # reconstruction
    for i in range(n):
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.gray()
        plt.axis("off")

    plt.show()

In [ ]:
# Autoencoding example (iterative, minimal, clic) FREEZES :-(

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split

img_pixels = 28*28
HIDDEN_SIZE = 32
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def normalize_img(image):
    #print(type(image['image']))
    processed_img = tf.cast(image['image'], tf.float32) / 255.
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    processed_img = tf.image.resize(processed_img, [28, 28])
    if processed_img.get_shape().as_list()[2] > 1:
        processed_img = tf.image.rgb_to_grayscale(processed_img) 
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    return processed_img

def get_image(image):
    return image, tf.reshape(image, (28*28, 1))

img_width = 28
img_height = 28
ITERATIONS = 20
for i in range(ITERATIONS):
    
    (ds_train, ds_test), ds_info = tfds.load(
        'clic',
        split=['train[:32]', 'test[:32]'],
        shuffle_files=True,
        #as_supervised=True, # Images are labeled
        with_info=True)
    
    ds_train = ds_train.map(normalize_img)
    ds_train = ds_train.map(get_image)
    ds_train = ds_train.cache() # Do not put me in the last position
    ds_train = ds_train.shuffle(
        ds_info.splits['train'].num_examples,
        reshuffle_each_iteration=True)
    ds_train = ds_train.batch(32)

    ds_test = ds_test.map(normalize_img)
    ds_test = ds_test.map(get_image)
    ds_test = ds_test.shuffle(
        ds_info.splits['test'].num_examples,
        reshuffle_each_iteration=True)
    ds_test = ds_test.batch(32)
    ds_test = ds_test.cache() # Put me in the last position

    print("len(train_DS) =", len(ds_train))
    print("len(test_DS) =", len(ds_test))
    
    n = len(ds_train)
    if n > 32:
        n = 32
        
    plt.figure(figsize=(40,2))
    
    for images in ds_train.take(1):
        print(len(images))
        for i in range(min(len(ds_train), n)):
            ax = plt.subplot(1, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    EPOCHS = 1
    model.fit(
        ds_train,
        epochs=EPOCHS,
        validation_data=ds_test)

    reconstructed_imgs = model.predict(ds_test)

    plt.figure(figsize=(40,4))

    # original
    for images in ds_test.take(1):
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            try:
                image = images[0][i].numpy()
                image *= 255
                image = image.astype("uint8").reshape(img_height, img_width)
                plt.imshow(image)
                plt.gray()
                plt.axis("off")
            except:
                pass

    # reconstruction
    for i in range(n):
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
        plt.gray()
        plt.axis("off")

    plt.show()

In [ ]:
# Using clic images (freezes)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
import time

HIDDEN_SIZE = 256
img_pixels = 64*64

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def process_dataset(image):
    #print(type(image['image']))
    processed_img = tf.cast(image['image'], tf.float32) / 255.
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    processed_img = tf.image.resize(processed_img, [64, 64])
    if processed_img.get_shape().as_list()[2] > 1:
        processed_img = tf.image.rgb_to_grayscale(processed_img) 
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    return processed_img

def define_y(image):
    return image, tf.reshape(image, (64*64, 1))

EPOCHS = 1
ITERATIONS = 100
for i in range(ITERATIONS):
    print("iteration", i)
    (ds_train, ds_test), ds_info = tfds.load(
        'clic',
        split=['train[:128]', 'test[:16]'],
        shuffle_files=True, # Shuffle the files between each epoch.
        with_info=True)

    ds_train = ds_train.map(process_dataset)
    #ds_train = ds_train.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.map(define_y)
    #ds_train = ds_train.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    #ds_train = ds_train.cache()
    #ds_train = ds_train.shuffle(
    #    ds_info.splits['train'].num_examples,
    #    reshuffle_each_iteration=True)
    ds_train = ds_train.batch(32)
    #ds_train = ds_train.batch(32, drop_remainder=True)
    #ds_train = ds_train.batch(1024, drop_remainder=True)
    #ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
    
    ds_test = ds_test.map(process_dataset)
    #ds_test = ds_test.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.map(define_y)
    #ds_test = ds_test.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(32)
    #ds_test = ds_test.batch(1024, drop_remainder=True)
    #ds_test = ds_test.cache()
    #ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

    model.fit(
        ds_train,
        epochs=EPOCHS,#//10#,
        #shuffle='batch',
        #verbose=2,
        validation_data=ds_test
    )
    #model.save("model")
    #model.load("model")
    
    reconstructed_imgs = model.predict(ds_test)

    n = 10
    plt.figure(figsize=(40,4))

    # original
    for images in ds_test.take(1):
        print(len(images))
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            image = images[0][i].numpy()
            image *= 255
            image = image.astype("uint8").reshape(64, 64)
            plt.imshow(image)
            plt.gray()
            plt.axis("off")

    # reconstruction
    for i in range(n):
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(reconstructed_imgs[i].reshape(64, 64))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

    plt.show()


In [ ]:
# Using clic images (save and load)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
import time

HIDDEN_SIZE = 256
img_pixels = 64*64

#input_layer = keras.Input(shape=(64*64,))
#hidden_layer = layers.Dense(HIDDEN_SIZE, activation="relu")(input_layer)
#output_layer = layers.Dense(img_pixels, activation="sigmoid")(hidden_layer)
#model = keras.Model(input_layer, output_layer)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def process_dataset(image):
    #print(type(image['image']))
    processed_img = tf.cast(image['image'], tf.float32) / 255.
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    processed_img = tf.image.resize(processed_img, [64, 64])
    if processed_img.get_shape().as_list()[2] > 1:
        processed_img = tf.image.rgb_to_grayscale(processed_img)
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    #processed_img = tf.image.resize(processed_img, [64*64, 1])
    return processed_img

def define_y(image):
    return image, tf.reshape(image, (64*64, 1))

EPOCHS = 1
ITERATIONS = 100
for i in range(ITERATIONS):
    print("iteration", i)
    (ds_train, ds_test), ds_info = tfds.load(
        'clic',
        split=['train[:128]', 'test[:16]'],
        shuffle_files=True, # Shuffle the files between each epoch.
        with_info=True)

    ds_train = ds_train.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(
        ds_info.splits['train'].num_examples,
        reshuffle_each_iteration=True)
    ds_train = ds_train.batch(32)
    #ds_train = ds_train.batch(1024, drop_remainder=True)
    ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
    
    ds_test = ds_test.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(32)
    #ds_test = ds_test.batch(1024, drop_remainder=True)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

    model.fit(
        ds_train,
        epochs=EPOCHS,#//10#,
        #shuffle='batch',
        validation_data=ds_test
    )
    keras.models.save_model(model, "model")
    
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(64, 64)),
      tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
      tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss="binary_crossentropy"
    )

    model = keras.models.load_model("model")

    #model.load("model")
    
    reconstructed_imgs = model.predict(ds_test)

    n = 10
    plt.figure(figsize=(40,4))

    # original
    for images in ds_test.take(1):
        print(len(images))
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            image = images[0][i].numpy()
            image *= 255
            image = image.astype("uint8").reshape(64, 64)
            plt.imshow(image)
            plt.gray()
            plt.axis("off")

    # reconstruction
    for i in range(n):
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(reconstructed_imgs[i].reshape(64, 64))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

    plt.show()


In [ ]:
# Using clic images (using a custom callback to visualize the training dataset after each epoch)

import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import Sequence
from skimage.io import imread
import math
from sklearn.model_selection import train_test_split
import time

HIDDEN_SIZE = 256
img_pixels = 64*64

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

def process_dataset(image):
    #print(type(image['image']))
    processed_img = tf.cast(image['image'], tf.float32) / 255.
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    processed_img = tf.image.resize(processed_img, [64, 64])
    if processed_img.get_shape().as_list()[2] > 1:
        processed_img = tf.image.rgb_to_grayscale(processed_img) 
    #print(processed_img.get_shape().as_list(), processed_img.dtype)
    return processed_img

def define_y(image):
    return image, tf.reshape(image, (64*64, 1))

EPOCHS = 100

def load_data():
    (ds_train, ds_test), ds_info = tfds.load(
        'clic',
        split=['train[:10]', 'test[:10]'],
        shuffle_files=True, # Shuffle the files between each epoch.
        with_info=True)

    ds_train = ds_train.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(
        ds_info.splits['train'].num_examples,
        reshuffle_each_iteration=True)
    ds_train = ds_train.batch(10)
    #ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

    n = 10
    for images in ds_train.take(1):
        print(len(images))
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            image = images[0][i].numpy()
            image *= 255
            image = image.astype("uint8").reshape(64, 64)
            plt.imshow(image)
            plt.gray()
            plt.axis("off")
    plt.show()

    ds_test = ds_test.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(10)
    ds_test = ds_test.cache()
    #ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
    return ds_train, ds_test
    
ds_train, ds_test = load_data()

class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        load_data()
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

model.fit(
    ds_train,
    epochs=EPOCHS,#//10#,
    shuffle='batch',
    validation_data=ds_test,
    callbacks=[CustomCallback()]
)

reconstructed_imgs = model.predict(ds_test)

n = 10
plt.figure(figsize=(40,4))

# original
for images in ds_test.take(1):
    print(len(images))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        image = images[0][i].numpy()
        image *= 255
        image = image.astype("uint8").reshape(64, 64)
        plt.imshow(image)
        plt.gray()
        plt.axis("off")

# reconstruction
for i in range(n):
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(reconstructed_imgs[i].reshape(64, 64))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()


In [ ]:

print(tf.__version__)
(ds_train, ds_test), ds_info = tfds.load(
    'clic',
    split=['train[:4%]', 'test[:1%]'],
    shuffle_files=True,
    with_info=True
)
print(ds_train, len(ds_train))

def process_dataset(image):
    print(type(image['image']))
    processed_img = tf.cast(image['image'], tf.float32) / 255.
    print(processed_img.get_shape().as_list(), processed_img.dtype)
    processed_img = tf.image.resize(processed_img, [64, 64])
    if processed_img.get_shape().as_list()[2] > 1:
        processed_img = tf.image.rgb_to_grayscale(processed_img) 
    print(processed_img.get_shape().as_list(), processed_img.dtype)
    return processed_img

def define_y(image):
    return image, tf.reshape(image, (64*64, 1))

#ds_train = ds_train.map(get_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(1024)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

plt.figure(figsize=(10, 10))
#class_names = ds_train.class_names

print("take=", ds_train.take(1))

for images in ds_train.take(1):
    print(len(images))
    for i in range(32):
        #print(len(images[i]))
        ax = plt.subplot(6, 6, i + 1)
        # plt.imshow(images[i].numpy().astype("uint8"))
        image = images[0][i].numpy()
        image *= 255
        image = image.astype("uint8").reshape(64,64)
        #print(image.shape)
        plt.imshow(image)
        plt.gray()
        #plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
print(ds_train, len(ds_train))

ds_test = ds_test.map(process_dataset, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(define_y, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(1024)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

print(ds_test, len(ds_test))

img_pixels = 64*64

In [ ]:
HIDDEN_SIZE = 256

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
  tf.keras.layers.Dense(img_pixels, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="binary_crossentropy"
)

model.summary()

In [ ]:
model.fit(ds_train,
    epochs=EPOCHS,
    validation_data=ds_test)

In [ ]:
reconstructed_imgs = model.predict(ds_test)

In [ ]:
len(reconstructed_imgs)

In [ ]:
reconstructed_imgs

In [ ]:
type(reconstructed_imgs)

In [ ]:
# Show some reconstructions.
n = 4

img_width = 64
img_height = 64

for images in ds_train.take(1):
    print(len(images))
    for i in range(32):
        #print(len(images[i]))
        ax = plt.subplot(6, 6, i + 1)
        # plt.imshow(images[i].numpy().astype("uint8"))
        image = images[0][i].numpy()
        image *= 255
        image = image.astype("uint8").reshape(64,64)
        #print(image.shape)
        plt.imshow(image)
        #plt.title(class_names[labels[i]])
        plt.gray()
        plt.axis("off")

for i in range(n):
  # reconstruction
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(reconstructed_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))> 60000
<PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None)}> 163

In [ ]:
64*64

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
    
<PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
    
<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
autoencoder.fit(train_DS, epochs=2)

In [ ]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label
train_DS = train_DS.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence
class CLIC_Sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)#.astype(np.int)
    
    def get_batch(batch_filenames):
        batch = []
        for filename in batch_filenames:
            image = imread(filename)
            resized_image = resize(image, (200, 200))
            image_01 = resized_image / 255.0
            batch.append(image_01)
        return batch

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        image = imread(file_name)
        resized_image = resize(image, (200, 200))
        return np.array(get_batch(batch_x)), np.array(batch_y)

In [ ]:

train_DS = tfds.as_numpy(train_DS) 
test_DS = tfds.as_numpy(test_DS)

x_train, y_train = train_DS["image"], train_DS["label"]
x_test, y_test = test_DS["image"], test_DS["label"]

In [ ]:
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

In [ ]:
encoding_dim = 32
img_height = x_train[0].shape[0]
img_width = x_train[0].shape[1]
img_pixels = img_width*img_height
print(img_pixels)

In [ ]:
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [ ]:
print(x_train.shape, x_test.shape)

In [ ]:
input_img = keras.Input(shape=(img_pixels,))
encoded = layers.Dense(encoding_dim, activation="relu")(input_img)
decoded = layers.Dense(img_pixels, activation="sigmoid")(encoded)
autoencoder = keras.Model(input_img, decoded)

In [ ]:
encoder = keras.Model(input_img, encoded)

In [ ]:
encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
autoencoder.fit(x_train, x_train, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_data=(x_test, x_test))

In [ ]:
encoded_imgs = encoder.predict(x_test)

In [ ]:
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(40,4))
for i in range(n):
  # original
  ax = plt.subplot(2, n, i+1)
  plt.imshow(x_test[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  
  # reconstruction
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(decoded_imgs[i].reshape(img_height, img_width))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
def _normalize_img(img, label):
  img = tf.cast(img, tf.float32) / 255.
  return (img, label)

train_DS = tfds.load('mnist', split='train', as_supervised=True)
train_DS = train_DS.batch(32)
train_DS = train_DS.map(_normalize_img)

test_DS = tfds.load('mnist', split='test', as_supervised=True)
test_DS = test_DS.batch(32)
test_DS = test_DS.map(_normalize_img)

#model.fit(ds_train, epochs=5)

In [ ]:
def _normalize_img(img, label):
  img = tf.cast(img, tf.float32) / 255.
  return (img, label)

train_DS = tfds.load('mnist', split='train', as_supervised=True)
train_DS = train_DS.batch(32)
train_DS = train_DS.map(_normalize_img)

test_DS = tfds.load('mnist', split='test', as_supervised=True)
test_DS = test_DS.batch(32)
test_DS = test_DS.map(_normalize_img)

#model.fit(ds_train, epochs=5)

In [ ]:
(train_DS, test_DS), info = tfds.load(name="mnist", split=[tfds.Split.TRAIN,tfds.Split.TEST], with_info=True, as_supervised=True)
train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples
print(train_size, test_size)

def _normalize_img(img, label):
  img = tf.cast(img, tf.float32) / 255.
  return (img, label)

train_DS = train_DS.batch(32)
train_DS = train_DS.map(_normalize_img)
test_DS = test_DS.batch(32)
test_DS = test_DS.map(_normalize_img)